### 步驟
#### 1. 讀取  地區資訊  和  東方果實蠅的東西
        out_put = di, df
#### 2. 進行 data_organization()
        out_put = X, y, 分級初始值, 級距, 級數
#### 3. 進行 data balancing
        out_put = X, y
#### 3.5 讀取欲預測之天氣()
        Out_put = 欲預測之天氣_X
#### 4. 重複十次以下步驟 :   尚未完成
        input = X, y, 欲預測之天氣, 害蟲名稱
        
        把data 八二分  計算 score 的值，儲存到buffer_scoring
        用 training data 預測   with   X = 輸入的天氣，儲存到buffer_predict
        
        svm(kernel = rbf, linear, poly, sigmoid)
        RandomForest(n_estimator = 20, 25, 30, 35)
#### 5. 計算
###### buffer_scoring 的平均與標準差
###### 如何選取預測值
        出現最多次的標籤
        把 scoring的平方 乘上 buffer_predict 除以 sum(scoring的平方)
        選取 top 7 scoring 的 prediction ，選取出現最多次的標籤
#### 6. 儲存資訊 in list 包括    害蟲名稱   縣市名稱   n天後預測   預測規模   誤差   初始值   級距   級數  

In [ ]:
def predict_weather(X, y, predict_X, DF):
    import statistics

    X = np.array(X)
    y = np.array(y)
    
    predict_X = np.array(predict_X)
        
    data = X
    score = []
    predict = []
    time = 10
    repeat = 10
    prediction = 0
    city_name = ''
    score_standard_deviation = 0
    buffer_X = X
    buffer_predict_X = predict_X
    #print(predict_X)
    
    for pests in ['東方果實蠅']:
        city_name = predict_X[0] + predict_X[1]
        
        for day in [1, 2, 3]:
            X = buffer_X
            predict_X = buffer_predict_X
            score=[]
            predict=[]
            #print(X[0])
            #print(predict_X)
            #print('predict_X')
            #print(X.shape)
            
            buffera_X=[]
            for row_X in X:
                buffera_X.append(row_X[:(10-day)*5])
            buffera_predict_X = predict_X[2:(2+(10-day)*5)]
            
            X = buffera_X
            predict_X = buffera_predict_X
            
            #print(X[0])
            #print(predict_X)
            #print('predict_X')
            X = np.array(X)
            #print(X.shape)
            #predict_X = predict_X.reshpae(1,-1)
            
            for val in range(30,30+time):
                clf = RandomForestClassifier(n_estimators = val)
                for repeata in range(repeat):
                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
                    score.append(clf.fit(X_train,y_train).score(X_test, y_test))
                    predict_X = predict_X.reshape(1,-1)
                    predict.append(clf.fit(X_train,y_train).predict(predict_X))

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
            buffer_score_svm = clf.fit(X_train,y_train).score(X_test, y_test)
            predict_X = predict_X.reshape(1,-1)
            buffer_predict_svm = clf.fit(X_train,y_train).predict(predict_X)
            for i in range(time*repeat):
                score.append(clf.fit(X_train,y_train).score(X_test, y_test))
                predict_X = predict_X.reshape(1,-1)
                predict.append(clf.fit(X_train,y_train).predict(predict_X))
            # 投票機制採用 score 平方加權
            #print('score')
            #print(score)
            #print('predict')
            #print(predict)
            weight = [i**2 for i in score]
            prediction = sum([weight[i]*predict[i] for i in range(len(score))])/sum(weight)
            score_standard_deviation = statistics.stdev(score)
            score = statistics.mean(score)
            DF2=pd.DataFrame([[pests, city_name, day, prediction, score, score_standard_deviation]], columns=column)
            DF = DF.append(DF2, ignore_index=True)
            #plot_box_RandomForest(DF,32)
            
            
    return DF


def predict_X(dia):
    buffer = []
    today_date_year, today_date_month, today_date_day = today_date()
    sub_buffer=[]
    
    #print('ttttt')
    if int(today_date_day) > 10:
        sub_buffer, need_to_continue =write_day10(dia, today_date_year, today_date_month, today_date_day)
        if need_to_continue == 1:
            print('there is a problem')
        else:
            buffer.append(sub_buffer)
    if int(today_date_day) < 10 and int(today_date_month) != 1:
        sub_buffer, need_to_continue =write_gap_month(dia, today_date_year, today_date_month, today_date_day)
        if need_to_continue == 1:
            print('there is a problem')
        else:
            buffer.append(sub_buffer)
    if int(today_date_day) < 10 and int(today_date_month) == 1:
        sub_buffer, need_to_continue =write_gap_year(dia, today_date_year, today_date_month, today_date_day)
        if need_to_continue == 1:
            print('there is a problem')
        else:
            buffer.append(sub_buffer)
    
    buffera = []
    for i in range(len(buffer[0])):
        if buffer[0][i] == '':
            buffera.append(0)
        elif buffer[0][i] == 'X':
            buffera.append(0)
        elif buffer[0][i] == 'T':
            buffera.append(0)
        elif i == 0 or i == 1:
                buffera.append(buffer[0][i])
        else:
            buffera.append(float(buffer[0][i]))
            
    return buffera


In [ ]:
import pandas as pd
import os 
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.decomposition import RandomizedPCA
from sklearn.learning_curve import learning_curve
from bs4 import BeautifulSoup
import datetime

import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt
import csv
from sklearn import svm
import numpy as np
from sklearn import neighbors
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
import math
import pylab as pl
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
import seaborn

print('成功')

In [ ]:
def read_data():#list comprehension
    with open('.\\data\\地區資訊new.csv', 'r') as f:  #opens PW file #她可以幫忙協助關閉程式
        reader = csv.reader(f)
        di = list(list(rec) for rec in csv.reader(f, delimiter=',')) #reads csv into a list of lists
        f.close()
    di.remove(di[0])
    return di

def data_organization(buffer, dia, n_start, n_multi, n_time):
    print('data_organization_',n_start,' ** ',n_multi, n_time)
    while [] in buffer:
        buffer.remove([])
    input_x=[]
    real_y=[]
    
    
    range_level = []
    range_level.append(0)
    for i in range(n_time):
        range_level.append(n_start*pow(n_multi,i))
    range_level[-1]=(10000)
    for buffera in buffer:
        buffera[0] = dia[0]
        buffera[1] = dia[1]
        buffera[2] = dia[2]        
        
        for i in range(n_time):
            if float(buffera[6]) < range_level[i+1] and float(float(buffera[6])) >= range_level[i]:
                buffera[6] = str(i)
        
        input_x.append(buffera[7:])
        real_y.append(float(buffera[6]))
        
    buffer_a = []
    buffer_b = []
    min_length=50
    max_length=30
    for element in input_x:
        if len(element) < min_length:
            min_length = len(element)
        if len(element) > max_length:
            max_length = len(element)
        for elementa in element:
            #print(type(elementa))
            if elementa == '':
                buffer_a.append(0)
            elif elementa == 'X' or elementa =='T':
                buffer_a.append(0)
            else:
                buffer_a.append(float(elementa))
        buffer_b.append(buffer_a)
        buffer_a = []
    for element in buffer_b:
        while len(element) != max_length:
            element.append(0)
    X = np.array(buffer_b)
    y = np.array(real_y)
    return X, y

print('成功')

def open_data(filename, dia, n_start, n_multi, n_time):

    print(filename)
    with open(filename,'r',encoding='utf-8', errors='ignore') as f:
        reader = csv.reader(f)
        buffer = list(list(rec) for rec in csv.reader(f, delimiter=',')) #reads csv into a list of lists
        f.close()
    
    X, y = data_organization(buffer, dia, n_multi, n_start, n_time)
    return X, y

def PCA_data(X,n_compo):
    print('running PCA with n_component = ' + str(n_compo))
    cov_mat = np.cov(X.T)
    eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)
    
    for i in range(len(eigen_vals)):
        if eigen_vals[i].imag != 0:
            np.delete(eigen_vals,i)
            np.delete(eigen_vecs,i)
    eigen_vals = eigen_vals.real
    
    tot = sum(eigen_vals)
    var_exp = [(i/tot) for i in sorted(eigen_vals, reverse=True)]
    cum_var_exp = np.cumsum(var_exp)
    
    counter = 0
    for i in cum_var_exp:
        counter = counter+1
        if i>0.8:
            Num_Pca_Over80 = counter
            break
    if n_compo == 'Pca_Over80':
        pca = RandomizedPCA(n_components=Num_Pca_Over80)
    else:
        pca = RandomizedPCA(n_components=2)
    X_pca = pca.fit_transform(X)
    
    return X_pca
    

In [ ]:
def rawX_to_plotbox(X, y, n_cv, train_model,weight,days):
    
    if days == 7:
        for row_X in X:
            #print(row_X)
            #print(type(row_X))
            #print(row_X.shape)
            np.delete(row_X, np.s_[-15:])
            
    if days == 8:
        for row_X in X:
            np.delete(row_X, np.s_[-10:])
    if days == 9:
        for row_X in X:
            np.delete(row_X, np.s_[-5:])

    data = X
    k = pd.DataFrame(data=data[0:,0:],    # values
                  index=range(len(data[0:])),    # 1st column as index
                  columns=range(len(data[0]))) 


    if train_model == 'RandomForest':
        scores = []
        DF = pd.DataFrame(columns=list('VS'))
        #-------------------RandomForset
        n = 30
        for val in range(1,n):
            if weight == 'None':
                clf = RandomForestClassifier(n_estimators = val,class_weight=None)
            else:
                clf = RandomForestClassifier(n_estimators = val,class_weight='balanced')
            try:
                validated = cross_val_score(clf, data, y, cv=n_cv)
            except:
                print('Error')
            #else:
            #    print('能怎麼辦呢??')
                
            #print(y[-10:])
            for count_cv in range(1,n_cv):
                DF2=pd.DataFrame([[val, validated[count_cv]]], columns=list('VS'))
                DF = DF.append(DF2, ignore_index=True)
        #--------------------SVM
        clf = SVC(class_weight='balanced')
        try:
            validated = cross_val_score(clf, data, y, cv=n_cv)
            #print(clf.fit(X[:-10],y[:-10]).score(X[:-10],y[:-10]), accuracy_score(clf.fit(X[:-10],y[:-10]).predict(X[-10:]),y[-10:]))
        except:
            print('???error')
            print(y)
            print('你覺得應該怎麼做呢??')
            return
            
        for count_cv in range(1,n_cv):
            DF2=pd.DataFrame([[50, validated[count_cv]]], columns=list('VS'))
            DF = DF.append(DF2, ignore_index=True)
            
        plt.title('RandomForest')
        plot_box_RandomForest(DF,n)
    else:
        print('fail to draw, please re examine your input')

print('完成')

In [ ]:
def ration_oo(y):
    y = np.array(y).astype(int)
    uni_class = np.unique(y)
    uni_class_voting = np.bincount(y)
    
    index = (uni_class_voting == 0)
    buffer = []
    for i in range(len(uni_class_voting)):
        if not index[i]:
            buffer.append(uni_class_voting[i])
    uni_class_voting = np.array(buffer)
    
    max_index = np.argmax(uni_class_voting)
    min_index = np.argmin(uni_class_voting)

    ratio = uni_class_voting[max_index]/uni_class_voting[min_index]
    ratio = int(round(ratio))
    
    return uni_class, uni_class_voting, max_index, min_index, ratio
def ration(y):
    y = np.array(y).astype(int)
    uni_class = np.unique(y)
    uni_class_voting = np.bincount((y))
    
    index = (uni_class_voting == 0)
    buffer = []
    for i in range(len(uni_class_voting)):
        if not index[i]:
            buffer.append(uni_class_voting[i])
    uni_class_voting = np.array(buffer)
    
    max_index = np.argmax(uni_class_voting)
    min_index = np.argmin(uni_class_voting)

    ratio = uni_class_voting[max_index]/uni_class_voting[min_index]
    ratio = int(round(ratio))
    
    return ratio
print('成功')

In [ ]:
def write_gap_year(row, year, month, day):
    need_to_continue = 0
    day_10_ago = ((datetime.datetime.now()-datetime.timedelta(days=10)).strftime("%d"))
    today_date_year, today_date_month, today_date_day = today_date()
    
    filename_last_month = '.\\data\\' + row[0] + row[1] + str(int(year)-1) + str(int(month)-1) + '.txt' 
    row = row[:2]
    with open(filename_last_month,'r') as f:
        data = f.read()
        f.close()
    sp = BeautifulSoup(data,'lxml')
    tra = sp.find_all('tr')
    tra.remove(tra[0])
    if tra ==[]:
        print('tra=[]')
        need_to_continue = 1
    tra.remove(tra[0])
    if tra ==[]:
        print('tra=[]')
        need_to_continue = 1
    tra.remove(tra[0])
    if tra ==[]:
        print('tra=[]')
        need_to_continue = 1
    traa_count=0
    for traa in tra:
        traa_count = traa_count+1
        if traa_count >= int(day_10_ago):
            tda=traa.find_all('td')
            tdaa_count = 0
            for tdaa in tda:
                tdaa_count = tdaa_count +1
                if  tdaa_count == 1:
                    row.append(tdaa.text)
                if  tdaa_count == 8 or tdaa_count==9 or tdaa_count==11 or tdaa_count==18:
                    row.append(tdaa.text.replace('\xa0',''))
                    
    filename_last_month = '.\\data\\' + row[0] + row[1] + year + str(int(month)-1) + '.txt' 
    with open(filename_last_month,'r') as f:
        data = f.read()
        f.close()
    sp = BeautifulSoup(data,'lxml')
    tra = sp.find_all('tr')
    tra.remove(tra[0])
    if tra ==[]:
        print('tra=[]')
        need_to_continue = 1
    tra.remove(tra[0])
    if tra ==[]:
        print('tra=[]')
        need_to_continue = 1
    tra.remove(tra[0])
    if tra ==[]:
        print('tra=[]')
        need_to_continue = 1
    traa_count=0
    for traa in tra:
        traa_count = traa_count+1
        if traa_count < int(day):
            tda=traa.find_all('td')
            tdaa_count = 0
            for tdaa in tda:
                tdaa_count = tdaa_count +1
                if  tdaa_count == 1:
                    row.append(tdaa.text)
                if  tdaa_count == 8 or tdaa_count==9 or tdaa_count==11 or tdaa_count==18:
                    row.append(tdaa.text.replace('\xa0',''))
    
    print('成功')
    return row, need_to_continue

def write_gap_month(row, year, month, day):
    need_to_continue = 0
    day_10_ago = ((datetime.datetime.now()-datetime.timedelta(days=10)).strftime("%d"))
    today_date_year, today_date_month, today_date_day = today_date()
    
    filename_last_month = '.\\data\\' + row[0] + row[1] + year + str(int(month)-1) + '.txt' 
    row = row[:2]
    with open(filename_last_month,'r') as f:
        data = f.read()
        f.close()
    sp = BeautifulSoup(data,'lxml')
    tra = sp.find_all('tr')
    tra.remove(tra[0])
    if tra ==[]:
        print('tra=[]')
        need_to_continue = 1
    tra.remove(tra[0])
    if tra ==[]:
        print('tra=[]')
        need_to_continue = 1
    tra.remove(tra[0])
    if tra ==[]:
        print('tra=[]')
        need_to_continue = 1
    traa_count=0
    for traa in tra:
        traa_count = traa_count+1
        if traa_count >= int(day_10_ago):
            tda=traa.find_all('td')
            tdaa_count = 0
            for tdaa in tda:
                tdaa_count = tdaa_count +1
                if  tdaa_count == 1:
                    row.append(tdaa.text)
                if  tdaa_count == 8 or tdaa_count==9 or tdaa_count==11 or tdaa_count==18:
                    row.append(tdaa.text.replace('\xa0',''))
                    
    filename_last_month = '.\\data\\' + row[0] + row[1] + year + str(int(month)-1) + '.txt' 
    with open(filename_last_month,'r') as f:
        data = f.read()
        f.close()
    sp = BeautifulSoup(data,'lxml')
    tra = sp.find_all('tr')
    tra.remove(tra[0])
    if tra ==[]:
        print('tra=[]')
        need_to_continue = 1
    tra.remove(tra[0])
    if tra ==[]:
        print('tra=[]')
        need_to_continue = 1
    tra.remove(tra[0])
    if tra ==[]:
        print('tra=[]')
        need_to_continue = 1
    traa_count=0
    for traa in tra:
        traa_count = traa_count+1
        if traa_count < int(day):
            tda=traa.find_all('td')
            tdaa_count = 0
            for tdaa in tda:
                tdaa_count = tdaa_count +1
                if  tdaa_count == 1:
                    row.append(tdaa.text)
                if  tdaa_count == 8 or tdaa_count==9 or tdaa_count==11 or tdaa_count==18:
                    row.append(tdaa.text.replace('\xa0',''))
    
    print('成功')
    return row, need_to_continue



def write_day10(row, year, month, day):
    need_to_continue = 0
    day_10_ago = ((datetime.datetime.now()-datetime.timedelta(days=10)).strftime("%d"))
    today_date_year, today_date_month, today_date_day = today_date()
    filename = '.\\data\\' + row[0] + row[1] + year + month + '.txt' 
    row = row[:2]
    with open(filename,'r') as f:
        data = f.read()
        f.close()
    sp = BeautifulSoup(data,'lxml')
    tra = sp.find_all('tr')
    tra.remove(tra[0])
    if tra ==[]:
        print('tra=[]')
        need_to_continue = 1
    tra.remove(tra[0])
    if tra ==[]:
        print('tra=[]')
        need_to_continue = 1
    tra.remove(tra[0])
    if tra ==[]:
        print('tra=[]')
        need_to_continue = 1
    traa_count=0
    
    for traa in tra:
        traa_count = traa_count+1
        if traa_count < int(today_date_day) and traa_count>= int(today_date_day)-10 :
            tda=traa.find_all('td')
            tdaa_count = 0
            for tdaa in tda:
                tdaa_count = tdaa_count +1
                if  tdaa_count == 1:
                    row.append(tdaa.text)
                if  tdaa_count == 8 or tdaa_count==9 or tdaa_count==11 or tdaa_count==18:
                    row.append(tdaa.text.replace('\xa0',''))
    print('成功')
    return row, need_to_continue


def today_date():
    import time
    ## dd/mm/yyyy格式
    today_date_year = time.strftime("%Y")
    today_date_month = time.strftime("%m").strip('0')
    today_date_day = time.strftime("%d")
    today_date = today_date_year + today_date_month + today_date_day
    return today_date_year, today_date_month, today_date_day
print('成功')

In [ ]:
di = read_data()
count_round = 0
di_count = 0

#column = ['病蟲害','縣市名稱','時間','規模','準確率','準確率誤差']
#DF = pd.DataFrame(columns=column)
for pests_name in ['東方果實蠅','瓜實蠅','甜菜夜蛾','斜紋夜蛾']:
    for dia in di:
        di_count = di_count +1
        filename = ('.\\data\\'+pests_name+ dia[0]+ dia[1]+'.csv')

        if di_count <= 35:
            continue

        if os.path.isfile(filename):

            X, y = open_data(filename, dia, 4, 4, 3)
            X, y = make_data_balanced(X,y)
            predicta = predict_X(dia)
            DF = predict_weather(X, y, predicta, DF)
            #plot_box_RandomForest(DF,n)
        
        #if di_count <= 15:
        #    print(di_count)
        #    break

            print(filename)
            print(datetime.datetime.now())

In [ ]:
X, y = open_data(filename, dia, 4, 4, 3)
a,b,c,d,e=ration_oo(y)
print(a,b)
print(ration(y))
X, y = make_data_balanced_2(X,y)
a,b,c,d,e=ration_oo(y)
print(a,b)
print(ration(y))

In [ ]:
def make_data_balanced(X,y):
    X = np.array(X)
    y = np.array(y)
    uni_class = np.unique(y)
    
    buffer_X =[]
    buffer_y =[]
    if len(uni_class) == 2:
        buffer_X,buffer_y = make_data_balanced_2(X,y)
    elif len(uni_class) == 1:
        print('天啊，我竟然也遇到這個狀況......那就不處理，直接送出去')
        buffer_X = X
        buffer_y = y
    else:
        ratio = ration(y)
        X = X.tolist()
        y = y.tolist()
        buffer_X = X
        buffer_y = y
        count_ratio = 0
        while ratio > 2.1:
            
            count_ratio += 1
            uni_class, uni_class_voting, max_index, min_index, ratio = ration_oo(buffer_y)
            print(uni_class, uni_class_voting)
            print(uni_class, uni_class_voting)
            for i in range(len(buffer_y)):
                if y[i] == uni_class[min_index]:
                    for repeat in range(round(ratio)):
                        buffer_X.append(X[i]) 
                        buffer_y.append(y[i])
            if count_ratio == 20:
                print('something happen')
                break
            
            ratio = ration(buffer_y)
            print(ratio)

    buffer_X = np.array(buffer_X)
    buffer_y = np.array(buffer_y)
    return buffer_X, buffer_y
            
def make_data_balanced_2(X,y):
    y = np.array(y)
    X = np.array(X)
    uni_class, uni_class_voting, max_index, min_index, ratio = ration_oo(y)
    if len(uni_class) == 2:#基本上會是2，除非不是透過make_data_balanced()呼叫
        # y的值
        # class的值 ============== uni_class
        # class的各數 ============ uni_class_voting
        # 最大與最小class的index == max
        ratio = ration(y)
        buffer_X = []
        buffer_y = []
        while ratio >= 2.1:
            for i in range(len(y)):
                if y[i] == uni_class[min_index]:
                    for repeat in range(int(ratio)):
                        buffer_X.append(X[i])
                        buffer_y.append(y[i])
                else:
                    buffer_X.append(X[i])
                    buffer_y.append(y[i])
            ratio = ration(buffer_y)
        buffer_X = np.array(buffer_X)
        buffer_y = np.array(buffer_y)
        
    else:
        buffer_X = X
        buffer_y = y
        
        
    return buffer_X, buffer_y

print('成功')